In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
import os
import joblib
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Set up paths
BASE_PATH = '/workspace/COMP-3608---PROJECT'
DATA_PATH = os.path.join(BASE_PATH, 'data')
MODELS_PATH = os.path.join(BASE_PATH, 'models')

In [ ]:
# Create models directory if it doesn't exist
if not os.path.exists(MODELS_PATH):
    os.makedirs(MODELS_PATH)

In [ ]:
# Function to plot confusion matrix
def plot_confusion_matrix(cm, class_names, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

In [ ]:
# Function to plot ROC curve
def plot_roc_curve(fpr, tpr, roc_auc, title):
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(title)
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
# Function to evaluate and compare models
def evaluate_models(X_train, X_test, y_train, y_test, dataset_name):
    # Create models
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\n{'-'*50}")
        print(f"Training and evaluating {name} on {dataset_name} dataset")
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Store results
        results[name] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'model': model,
            'predictions': y_pred,
            'probabilities': y_prob
        }
        
        # Print metrics
        print(f"\nModel: {name}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        
        # Print classification report
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        # Confusion Matrix
        cm = confusion_matrix(y_test, y_pred)
        plot_confusion_matrix(cm, ['Negative', 'Positive'], f'Confusion Matrix - {name} ({dataset_name})')
        
        # ROC Curve
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        plot_roc_curve(fpr, tpr, roc_auc, f'ROC Curve - {name} ({dataset_name})')
        
        # Save model
        model_filename = os.path.join(MODELS_PATH, f"{dataset_name}_{name.replace(' ', '_').lower()}.pkl")
        joblib.dump(model, model_filename)
        print(f"Model saved to {model_filename}")
    
    return results

In [ ]:
# Define helper function to prepare data for modeling
def prepare_data_for_modeling(df, target_column, categorical_cols=None, numerical_cols=None):
    if categorical_cols is None:
        categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    if numerical_cols is None:
        numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    # Remove target from feature lists if present
    if target_column in categorical_cols:
        categorical_cols.remove(target_column)
    if target_column in numerical_cols:
        numerical_cols.remove(target_column)
    
    # Create preprocessing pipeline
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ])
    
    # Split data
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    
    # Apply preprocessing
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    return X_train_processed, X_test_processed, y_train, y_test

In [ ]:
# DIABETES DATASET EVALUATION
print("Loading Diabetes Dataset...")
diabetes_df = pd.read_csv(os.path.join(DATA_PATH, 'feature_engineering/diabetes_feature_engineering'))

# Define categorical and numerical columns for diabetes
diabetes_cat_cols = ['gender', 'smoking_history', 'age_group', 'bmi_category', 'glucose_tolerance']
diabetes_num_cols = [col for col in diabetes_df.columns if col not in diabetes_cat_cols + ['diabetes']]

# Prepare data
X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes = prepare_data_for_modeling(
    diabetes_df, 'diabetes', diabetes_cat_cols, diabetes_num_cols)

# Evaluate models
diabetes_results = evaluate_models(X_train_diabetes, X_test_diabetes, y_train_diabetes, y_test_diabetes, 'diabetes')

In [ ]:
# 2. STROKE DATASET EVALUATION
print("\nLoading Stroke Dataset...")
stroke_df = pd.read_csv(os.path.join(DATA_PATH, 'feature_engineering/Stroke_feature_engineering'))

# For stroke dataset, many columns are already one-hot encoded
stroke_cat_cols = ['glucose_risk']
stroke_num_cols = [col for col in stroke_df.columns 
                  if col not in stroke_cat_cols + ['stroke', 'age_group'] 
                  and not col.startswith('gender_') 
                  and not col.startswith('ever_married_')
                  and not col.startswith('work_type_')
                  and not col.startswith('Residence_type_')
                  and not col.startswith('smoking_status_')]

# Prepare data
X_train_stroke, X_test_stroke, y_train_stroke, y_test_stroke = prepare_data_for_modeling(
    stroke_df, 'stroke', stroke_cat_cols, stroke_num_cols)

# Evaluate models
stroke_results = evaluate_models(X_train_stroke, X_test_stroke, y_train_stroke, y_test_stroke, 'stroke')